<a href="https://colab.research.google.com/github/jnsbrdbr/hopefully_last_thesis/blob/main/92_1000_PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import jaccard_score
from scipy.spatial.distance import pdist, squareform

from sklearn.decomposition import PCA
import copy

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **importing data**

In [3]:
import tables
tables.file._open_files.close_all()

In [4]:
store = pd.HDFStore('/content/drive/MyDrive/1000/Tdata.h5')

adj=store['adj']
store.close()

In [5]:
with pd.HDFStore('/content/drive/MyDrive/1000/TCFdf.h5',  mode='r') as newstore:
    CF = newstore.select('CFdf')

In [6]:
store = pd.HDFStore('/content/drive/MyDrive/1000/TTFdf.h5')
TF=store['TFdf']
store.close()

In [7]:
with pd.HDFStore('/content/drive/MyDrive/1000/TjaccardsimilaritydfR.h5',  mode='r') as newstore:
    jaccardsimilarity = newstore.select('jaccardsimilaritydfR')

In [8]:
store = pd.HDFStore('/content/drive/MyDrive/1000/TcosinesimilaritydfR.h5')
cosine=store['cosine_similaritydfR']

In [9]:
store = pd.HDFStore('/content/drive/MyDrive/1000/TL1normdf.h5')
L1_normdf=store['L1_normdf']
store.close()

In [10]:
store = pd.HDFStore('/content/drive/MyDrive/1000/Tsum.h5')
wsumdf=store['wsumdf']
store.close()

In [11]:
with pd.HDFStore('/content/drive/MyDrive/1000/Tmul.h5',  mode='r') as newstore:
    wmuldf = newstore.select('wmuldf')

In [12]:
with pd.HDFStore('/content/drive/MyDrive/1000/TLdata.h5',  mode='r') as newstore:
    adjl = newstore.select('adjl')

# **implementation of PCA**

In [25]:
pca = PCA(n_components=10)
CF_pca = pca.fit_transform(CF)
CF_pca_df=pd.DataFrame(CF_pca)

In [26]:
CF_pca_df

,0,1,2,3,4,5,6,7,8,9
0,-1.715972,-3.099936,-0.041249,-3.014462e-14,6.993068e-15,0.077295,1.691428,3.341114,2.942154e-13,-8.283869e-14
1,-1.715972,-3.099936,-0.041249,-1.077147e-13,-7.609726e-15,0.077295,1.691428,3.341114,-1.438132e-14,8.662557e-16
2,-1.715972,-3.099936,-0.041249,3.613522e-14,-6.634688e-14,0.077295,1.691428,3.341114,1.872817e-13,-2.918520e-14
3,-1.715972,-3.099936,-0.041249,-4.985799e-16,-1.950109e-14,0.077295,1.691428,3.341114,1.273288e-13,7.392117e-15
4,-1.715972,-3.099936,-0.041249,3.076934e-14,-2.010380e-14,0.077295,1.691428,3.341114,9.698404e-14,4.389324e-15
...,...,...,...,...,...,...,...,...,...,...
995,-2.114893,-4.572474,-0.754679,9.812903e-13,-5.991311e-14,8.797502,-0.661164,-0.063010,-3.763016e-02,1.747494e-02
996,-2.114893,-4.572474,-0.754679,9.808844e-13,-6.005569e-14,8.797502,-0.661164,-0.063010,1.491368e-02,4.994059e-02
997,-2.114893,-4.572474,-0.754679,9.811770e-13,-5.988958e-14,8.797502,-0.661164,-0.063010,-3.931630e-02,-1.973605e-02
998,-2.114893,-4.572474,-0.754679,9.807072e-13,-6.027299e-14,8.797502,-0.661164,-0.063010,1.801955e-02,8.807116e-02


In [27]:
pca = PCA(n_components=10)
TF_pca = pca.fit_transform(TF)
TF_pca_df=pd.DataFrame(TF_pca)

In [28]:
TF_pca_df.rename(columns = {0:10,1:11,2:12,3:13,4:14,5:15,6:16,7:17,8:18,9:19}, inplace = True)

In [29]:
TF_pca_df

,10,11,12,13,14,15,16,17,18,19
0,-26.167339,0.109748,0.187462,-0.926807,0.001555,2.380835e-11,6.914478e-11,-0.002291,-1.817205,-0.021183
1,-26.167339,0.109748,0.187462,-0.926807,0.001555,7.104211e-12,2.076908e-11,-0.002291,-1.817205,-0.021183
2,-26.167339,0.109748,0.187462,-0.926807,0.001555,-2.679553e-11,-7.939227e-11,-0.002291,-1.817205,-0.021183
3,-26.167339,0.109748,0.187462,-0.926807,0.001555,-4.077457e-12,-1.236655e-11,-0.002291,-1.817205,-0.021183
4,1393.385962,-11.935691,-29.759005,-0.507027,-0.476180,-4.284781e-13,6.729922e-13,0.884093,-1.749524,-0.183266
...,...,...,...,...,...,...,...,...,...,...
995,-29.238191,0.128642,0.210675,-1.647476,0.001097,3.597460e-13,8.232208e-14,-0.000867,-4.540979,-0.763937
996,-29.238191,0.128642,0.210675,-1.647476,0.001097,3.595939e-13,8.187443e-14,-0.000867,-4.540979,-0.763937
997,-29.238191,0.128642,0.210675,-1.647476,0.001097,3.602199e-13,8.370714e-14,-0.000867,-4.540979,-0.763937
998,-29.238191,0.128642,0.210675,-1.647476,0.001097,3.603274e-13,8.419083e-14,-0.000867,-4.540979,-0.763937


In [30]:
pca = PCA(n_components=10)
jaccard_similarity_pca = pca.fit_transform(jaccardsimilarity)
jaccard_similarity_pca_df=pd.DataFrame(jaccard_similarity_pca)

In [31]:
jaccard_similarity_pca_df.rename(columns = {0:20,1:21,2:22,3:23,4:24,5:25,6:26,7:27,8:28,9:29}, inplace = True)

In [32]:
jaccard_similarity_pca_df

,20,21,22,23,24,25,26,27,28,29
0,-1.715972,-3.099936,-0.041249,-4.448350e-14,2.745951e-14,0.077295,1.691428,3.341114,-3.030389e-14,-1.892928e-13
1,-1.715972,-3.099936,-0.041249,1.021530e-13,-4.251711e-14,0.077295,1.691428,3.341114,9.609947e-14,6.899815e-14
2,-1.715972,-3.099936,-0.041249,-6.938534e-15,-1.314966e-13,0.077295,1.691428,3.341114,2.632566e-14,4.538127e-14
3,-1.715972,-3.099936,-0.041249,-6.157188e-14,3.566145e-14,0.077295,1.691428,3.341114,-1.838835e-13,-2.847460e-13
4,-1.715972,-3.099936,-0.041249,1.288577e-14,-5.746930e-15,0.077295,1.691428,3.341114,9.324115e-15,3.456139e-14
...,...,...,...,...,...,...,...,...,...,...
995,-2.114893,-4.572474,-0.754679,5.219995e-13,2.167073e-13,8.797502,-0.661164,-0.063010,2.025037e-03,9.904048e-04
996,-2.114893,-4.572474,-0.754679,5.220249e-13,2.174647e-13,8.797502,-0.661164,-0.063010,1.436811e-02,-1.934106e-03
997,-2.114893,-4.572474,-0.754679,5.218006e-13,2.162754e-13,8.797502,-0.661164,-0.063010,-1.522981e-02,-2.362434e-02
998,-2.114893,-4.572474,-0.754679,5.222029e-13,2.175463e-13,8.797502,-0.661164,-0.063010,6.679121e-02,5.100611e-02


In [33]:
pca = PCA(n_components=10)
cosine_similarity_pca = pca.fit_transform(cosine)
cosine_similarity_pca_df=pd.DataFrame(cosine_similarity_pca)

In [34]:
cosine_similarity_pca_df.rename(columns = {0:30,1:31,2:32,3:33,4:34,5:35,6:36,7:37,8:38,9:39}, inplace = True)

In [35]:
cosine_similarity_pca_df

,30,31,32,33,34,35,36,37,38,39
0,-1.715972,-3.099936,-0.041249,-1.704887e-14,5.770425e-15,0.077295,1.691428,3.341114,1.544299e-13,-3.015257e-14
1,-1.715972,-3.099936,-0.041249,2.200372e-14,5.436645e-14,0.077295,1.691428,3.341114,2.811352e-14,3.499055e-14
2,-1.715972,-3.099936,-0.041249,-1.650612e-13,7.350938e-14,0.077295,1.691428,3.341114,4.294414e-14,1.199464e-13
3,-1.715972,-3.099936,-0.041249,1.435498e-13,-3.694812e-14,0.077295,1.691428,3.341114,-2.769225e-14,-9.795358e-14
4,-1.715972,-3.099936,-0.041249,-7.770230e-16,3.089996e-14,0.077295,1.691428,3.341114,9.701204e-15,-2.821274e-14
...,...,...,...,...,...,...,...,...,...,...
995,-2.114893,-4.572474,-0.754679,3.089626e-13,7.295118e-13,8.797502,-0.661164,-0.063010,2.100080e-02,-6.346292e-02
996,-2.114893,-4.572474,-0.754679,3.090394e-13,7.294617e-13,8.797502,-0.661164,-0.063010,-8.341401e-04,-3.731763e-02
997,-2.114893,-4.572474,-0.754679,3.103764e-13,7.285333e-13,8.797502,-0.661164,-0.063010,-9.139830e-02,-1.264879e-02
998,-2.114893,-4.572474,-0.754679,3.094543e-13,7.296640e-13,8.797502,-0.661164,-0.063010,-4.696902e-03,-2.891245e-02


In [36]:
pca = PCA(n_components=10)
L1_normdf_similarity_pca = pca.fit_transform(L1_normdf)
L1_normdf_similarity_pca_df=pd.DataFrame(L1_normdf_similarity_pca)

In [37]:
L1_normdf_similarity_pca_df.rename(columns = {0:40,1:41,2:42,3:43,4:44,5:45,6:46,7:47,8:48,9:49}, inplace = True)

In [38]:
L1_normdf_similarity_pca_df

,40,41,42,43,44,45,46,47,48,49
0,-1.715972,-3.099936,-0.041249,-4.204561e-14,-1.909008e-14,0.077295,1.691428,3.341114,-5.449817e-14,-9.195237e-14
1,-1.715972,-3.099936,-0.041249,-8.256112e-14,-4.623420e-14,0.077295,1.691428,3.341114,-1.493439e-14,3.060276e-14
2,-1.715972,-3.099936,-0.041249,-8.304640e-14,1.009972e-14,0.077295,1.691428,3.341114,-8.418185e-14,-9.703849e-14
3,-1.715972,-3.099936,-0.041249,-8.400154e-14,-8.916037e-15,0.077295,1.691428,3.341114,-7.593505e-14,-7.795276e-14
4,-1.715972,-3.099936,-0.041249,-4.345748e-14,2.704444e-15,0.077295,1.691428,3.341114,-2.579347e-14,6.612217e-14
...,...,...,...,...,...,...,...,...,...,...
995,-2.114893,-4.572474,-0.754679,-8.355032e-13,9.066742e-13,8.797502,-0.661164,-0.063010,-5.326168e-02,-1.038985e-02
996,-2.114893,-4.572474,-0.754679,-8.348619e-13,9.062363e-13,8.797502,-0.661164,-0.063010,-4.143557e-02,-1.909957e-02
997,-2.114893,-4.572474,-0.754679,-8.344489e-13,9.059014e-13,8.797502,-0.661164,-0.063010,-2.163935e-02,-1.933858e-03
998,-2.114893,-4.572474,-0.754679,-8.384774e-13,9.062034e-13,8.797502,-0.661164,-0.063010,2.917563e-02,-3.330915e-02


In [40]:
pca = PCA(n_components=10)
wsumdf_similarity_pca = pca.fit_transform(wsumdf)
wsumdf_similarity_pca_df=pd.DataFrame(wsumdf_similarity_pca)

In [41]:
wsumdf_similarity_pca_df.rename(columns = {0:50,1:51,2:52,3:53,4:54,5:55,6:56,7:57,8:58,9:59}, inplace = True)

In [42]:
wsumdf_similarity_pca_df

,50,51,52,53,54,55,56,57,58,59
0,-0.158046,5.930542e-02,3.907021e-02,1.099061e-01,-7.818182e-02,-1.845667e-02,-1.217298e-02,1.414027e-02,3.376248e-02,4.533536e-02
1,-0.158046,-1.227267e-02,-3.426230e-02,6.324133e-02,-2.936951e-02,-1.735494e-02,1.341969e-02,4.464113e-02,-1.511709e-02,4.681900e-02
2,-0.158046,1.353342e-02,-8.483484e-02,-8.488620e-02,-2.124233e-02,1.776292e-02,-5.523460e-02,6.972080e-02,1.926976e-02,-1.111527e-02
3,-0.158046,-9.840913e-02,-5.913124e-02,-1.055238e-02,3.451212e-02,6.612345e-02,1.543536e-02,1.048965e-01,6.300173e-02,-9.753092e-03
4,17.554873,1.486561e-12,-5.432688e-13,1.441318e-13,1.597114e-12,-4.659493e-13,1.441996e-12,-1.073018e-12,-1.217068e-12,-5.112559e-13
...,...,...,...,...,...,...,...,...,...,...
995,-0.158046,-2.132079e-02,2.847706e-02,-8.962128e-04,7.341290e-02,-3.662986e-02,7.954443e-02,-8.112078e-02,-3.331378e-02,-1.443683e-02
996,-0.158046,4.194744e-02,3.268484e-02,-3.701662e-02,5.149061e-02,-2.116510e-02,3.327137e-02,-3.776965e-02,-7.961523e-02,4.955824e-02
997,-0.158046,2.834609e-02,-9.023845e-02,2.225049e-02,-2.363142e-02,-4.857143e-02,4.664053e-02,1.688479e-02,-3.183880e-02,5.433903e-03
998,-0.158046,-5.466336e-02,-1.421375e-02,1.004836e-01,-1.258185e-03,-6.381831e-02,-5.673184e-03,-1.804906e-02,-1.785241e-02,-7.210815e-02


In [43]:
pca = PCA(n_components=10)
wmuldf_similarity_pca = pca.fit_transform(wmuldf)
wmuldf_similarity_pca_df=pd.DataFrame(wmuldf_similarity_pca)

In [44]:
wmuldf_similarity_pca_df.rename(columns = {0:60,1:61,2:62,3:63,4:64,5:65,6:66,7:67,8:68,9:69}, inplace = True)

In [45]:
wmuldf_similarity_pca_df

,60,61,62,63,64,65,66,67,68,69
0,-0.111385,2.773322e-04,9.491344e-03,-7.804946e-03,-5.670672e-03,2.927722e-03,1.908601e-02,-1.444934e-02,-2.196036e-02,-7.038974e-03
1,-0.111385,-8.023104e-03,-2.400882e-03,-4.764501e-04,3.001035e-03,2.223896e-02,-6.529277e-03,-9.276991e-03,-1.871558e-02,4.280893e-03
2,-0.111385,-6.572360e-04,1.802349e-02,-8.207065e-03,8.771168e-03,-4.267901e-03,3.784342e-05,-7.928601e-03,2.407668e-02,1.858991e-03
3,-0.111385,1.562232e-02,1.669297e-02,6.232700e-03,-5.914754e-03,-2.125491e-02,5.786478e-03,-1.987696e-02,-1.467211e-02,-1.827221e-02
4,12.371234,1.615195e-15,1.979979e-14,-1.314080e-14,-5.936133e-16,-1.114754e-13,2.508360e-14,-1.752908e-14,-6.587185e-15,-6.375053e-15
...,...,...,...,...,...,...,...,...,...,...
995,-0.111385,-1.163868e-02,-6.831592e-03,-2.390951e-02,-6.128685e-04,2.083087e-02,6.098501e-03,1.520041e-02,-5.489348e-03,-1.505644e-02
996,-0.111385,3.774208e-03,1.141794e-02,-3.274494e-02,1.119192e-02,-2.678543e-02,-9.823778e-03,1.566019e-02,3.322733e-02,-9.037160e-03
997,-0.111385,8.641479e-03,-3.511065e-03,3.228450e-02,-4.198955e-03,1.000159e-03,-4.999201e-02,7.336873e-03,-1.289204e-02,-1.050892e-02
998,-0.111385,-3.519257e-02,1.069808e-02,5.276833e-03,1.659729e-02,3.791869e-02,1.136222e-02,2.956178e-02,1.088086e-02,-1.952860e-03


In [46]:
y=adjl.label
y_arr=np.array(y)
y_df=pd.DataFrame(y_arr)

In [47]:
y_df.rename(columns = {0:70}, inplace = True)

In [48]:
#concat
frames = [CF_pca_df, TF_pca_df, jaccard_similarity_pca_df,cosine_similarity_pca_df,
          L1_normdf_similarity_pca_df,wsumdf_similarity_pca_df,wmuldf_similarity_pca_df,y_df]
result = pd.concat(frames,axis=1)

In [50]:
result

,0,1,2,3,4,5,6,7,8,9,...,61,62,63,64,65,66,67,68,69,70
0,-1.715972,-3.099936,-0.041249,-3.014462e-14,6.993068e-15,0.077295,1.691428,3.341114,2.942154e-13,-8.283869e-14,...,2.773322e-04,9.491344e-03,-7.804946e-03,-5.670672e-03,2.927722e-03,1.908601e-02,-1.444934e-02,-2.196036e-02,-7.038974e-03,1
1,-1.715972,-3.099936,-0.041249,-1.077147e-13,-7.609726e-15,0.077295,1.691428,3.341114,-1.438132e-14,8.662557e-16,...,-8.023104e-03,-2.400882e-03,-4.764501e-04,3.001035e-03,2.223896e-02,-6.529277e-03,-9.276991e-03,-1.871558e-02,4.280893e-03,1
2,-1.715972,-3.099936,-0.041249,3.613522e-14,-6.634688e-14,0.077295,1.691428,3.341114,1.872817e-13,-2.918520e-14,...,-6.572360e-04,1.802349e-02,-8.207065e-03,8.771168e-03,-4.267901e-03,3.784342e-05,-7.928601e-03,2.407668e-02,1.858991e-03,1
3,-1.715972,-3.099936,-0.041249,-4.985799e-16,-1.950109e-14,0.077295,1.691428,3.341114,1.273288e-13,7.392117e-15,...,1.562232e-02,1.669297e-02,6.232700e-03,-5.914754e-03,-2.125491e-02,5.786478e-03,-1.987696e-02,-1.467211e-02,-1.827221e-02,1
4,-1.715972,-3.099936,-0.041249,3.076934e-14,-2.010380e-14,0.077295,1.691428,3.341114,9.698404e-14,4.389324e-15,...,1.615195e-15,1.979979e-14,-1.314080e-14,-5.936133e-16,-1.114754e-13,2.508360e-14,-1.752908e-14,-6.587185e-15,-6.375053e-15,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-2.114893,-4.572474,-0.754679,9.812903e-13,-5.991311e-14,8.797502,-0.661164,-0.063010,-3.763016e-02,1.747494e-02,...,-1.163868e-02,-6.831592e-03,-2.390951e-02,-6.128685e-04,2.083087e-02,6.098501e-03,1.520041e-02,-5.489348e-03,-1.505644e-02,1
996,-2.114893,-4.572474,-0.754679,9.808844e-13,-6.005569e-14,8.797502,-0.661164,-0.063010,1.491368e-02,4.994059e-02,...,3.774208e-03,1.141794e-02,-3.274494e-02,1.119192e-02,-2.678543e-02,-9.823778e-03,1.566019e-02,3.322733e-02,-9.037160e-03,1
997,-2.114893,-4.572474,-0.754679,9.811770e-13,-5.988958e-14,8.797502,-0.661164,-0.063010,-3.931630e-02,-1.973605e-02,...,8.641479e-03,-3.511065e-03,3.228450e-02,-4.198955e-03,1.000159e-03,-4.999201e-02,7.336873e-03,-1.289204e-02,-1.050892e-02,1
998,-2.114893,-4.572474,-0.754679,9.807072e-13,-6.027299e-14,8.797502,-0.661164,-0.063010,1.801955e-02,8.807116e-02,...,-3.519257e-02,1.069808e-02,5.276833e-03,1.659729e-02,3.791869e-02,1.136222e-02,2.956178e-02,1.088086e-02,-1.952860e-03,1


In [51]:
with pd.HDFStore('/content/drive/MyDrive/last/last_result_1000',  mode='w') as store:
    store.append('result', result,format='table')